In [36]:
import importlib

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from apollo.settings import LONG_SIGNAL, NO_SIGNAL, SHORT_SIGNAL


In [37]:
# Manually reloading python module such that
# jupyter reflects changes without kernel restart

import apollo.api.yahoo_api_connector as yac
import apollo.calculations.key_reversals as kr
import apollo.calculations.average_true_range as atr
from apollo.utils.common import to_default_date_string

importlib.reload(yac)
importlib.reload(kr)
importlib.reload(atr);

In [38]:
ticker = "SPY"
start_date = "2023-09-01"
end_date = "2024-03-01"

api_connector = yac.YahooApiConnector(ticker, start_date, end_date)
dataframe = api_connector.request_or_read_prices()

dataframe;

In [39]:
WINDOW_SIZE = 15
VOLATILITY_MULTIPLIER = 0.5

kr_calculator = kr.KeyReversalsCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

atr_calculator = atr.AverageTrueRangeCalculator(
    dataframe=dataframe,
    window_size=WINDOW_SIZE,
)

kr_calculator.calculate_key_reversals()
atr_calculator.calculate_average_true_range()

dataframe.dropna(inplace=True)

dataframe["signal"] = NO_SIGNAL

long = (dataframe["kr"] == LONG_SIGNAL) & (
    dataframe["tr"] > dataframe["atr"] * VOLATILITY_MULTIPLIER
)
dataframe.loc[long, "signal"] = LONG_SIGNAL

short = (dataframe["kr"] == SHORT_SIGNAL) & (
    dataframe["tr"] > dataframe["atr"] * VOLATILITY_MULTIPLIER
)
dataframe.loc[short, "signal"] = SHORT_SIGNAL

dataframe

,ticker,open,high,low,close,adj close,volume,kr,tr,atr,signal
date,,,,,,,,,,,
2023-10-12,SPY,436.950012,437.339996,431.230011,433.660004,430.571075,81154200,-1.0,6.109985,6.140197,-1
2023-10-13,SPY,435.209991,436.450012,429.880005,431.500000,428.426422,95143100,0.0,6.570007,6.168851,0
2023-10-16,SPY,433.820007,437.140015,433.570007,436.040009,432.934143,75433200,0.0,8.713593,6.338500,0
2023-10-17,SPY,432.809998,438.140015,432.450012,436.019989,432.914246,75324700,-1.0,5.690002,6.295267,-1
2023-10-18,SPY,434.190002,435.179993,429.089996,430.209991,427.145630,93559800,0.0,6.089996,6.281582,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-02-23,SPY,509.269989,510.130005,507.100006,507.850006,506.276978,61321800,0.0,4.201935,5.368945,0
2024-02-26,SPY,508.299988,508.750000,505.859985,505.989990,504.422729,50386700,0.0,2.890015,5.203683,0
2024-02-27,SPY,506.700012,507.160004,504.750000,506.929993,505.359833,48854500,0.0,2.737274,5.039256,0


In [40]:
prime_value = "close"
trace_value = "signal"

x = dataframe.index.to_numpy()

y1 = dataframe[prime_value].to_numpy()
y2 = dataframe[trace_value].to_numpy()

# Create the first trace with the primary y-axis
trace1 = go.Scatter(x=x, y=y1, name=prime_value)

# Create the second trace with the secondary y-axis
trace2 = go.Scatter(x=x, y=y2, name=f"{trace_value}", yaxis="y2")

# Plot title
title = (
    f"{ticker}:"
    f" {to_default_date_string(dataframe.index.to_numpy()[0])}"
    f" - {to_default_date_string(dataframe.index.to_numpy()[-1])}"
)

# Create the layout with two y-axes
layout = go.Layout(
    title=title,
    yaxis={},
    yaxis2={"overlaying": "y", "side": "right"},
    height=650,
)

# Create the figure and add traces to it
fig = go.Figure(data=[trace1, trace2], layout=layout)

fig.update_xaxes(
    showspikes=True,
    spikemode="across",
    spikecolor="black",
    spikethickness=0.5,
)